### Install the libraries

In [1]:
# !pip install pinecone llama-index-vector-stores-pinecone llama-index-core llama-index-embeddings-huggingface -q
# pip install -U llama-index-core llama-index-embeddings-huggingface
!pip install -U langchain langchain-core langchain-openai langgraph python-dotenv openai pandas llama-index llama-index-vector-stores-pinecone llama-index-core llama-index-embeddings-huggingface llama-index-llms-groq



  Using cached openai-2.6.1-py3-none-any.whl.metadata (29 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.1.1
    Uninstalling python-dotenv-1.1.1:
      Successfully uninstalled python-dotenv-1.1.1



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import the libraries

In [35]:
from llama_index.core import(
    Settings, 
    VectorStoreIndex, 
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
import time

### Initialize the APIs

In [36]:
# loading the .env file
load_dotenv()

# initializing Groq and Pinecone APIs
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key = PINECONE_API_KEY)

### LLM and Embedding Model Configuration

In [37]:
llm = Groq(model = "llama-3.1-8b-instant")
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

2025-10-27 11:28:06,849 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


### Pinecone Vector DB Creation

In [38]:
# setting index_name
index_name = "llama3-groq-pinecone"

# checking existing indexes
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# creating an index if it doesn't exists
if index_name not in existing_indexes: 
    pc.create_index(
        name = index_name, 
        dimension = 768, 
        metric = "cosine", 
        spec = ServerlessSpec(
                cloud = "aws", 
                region = "us-east-1"
        )
    )

# connecting to the created index
index = pc.Index(index_name)
time.sleep(1)

# describing index statistics
index.describe_index_stats()



{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 44}},
 'total_vector_count': 44,
 'vector_type': 'dense'}

### Loading the data

In [39]:
documents = documents = SimpleDirectoryReader("store_offers_data").load_data()

### Using Pinecone Vector Store

In [40]:
vector_store = PineconeVectorStore(pinecone_index = index)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

Upserted vectors: 100%|██████████| 6/6 [00:01<00:00,  3.76it/s]


### Querying the stored data

In [43]:
question = f"Are there any offers in October for groceries, If yes tell me"

query_engine = index.as_query_engine()
response = query_engine.query(f"{question}")

2025-10-27 11:29:27,170 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [44]:
print(response)

Yes, there are offers in October for groceries. 

The Autumn Harvest Festival is available from October 1 to October 31, offering up to 15% off select products. The Harvest Your Favorites promotion is available from October 15 to November 15, offering 10% off when you mix and match your favorite harvest snacks. The Savor the Season Sale is available from October 25 to November 10, offering the deliciousness of FrostyHarvest Seasonal Comfort Soup Mix and AutumnHarvest Crisp Apples.
